In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
'Obtain the MNist dataset'
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',version=1)
X,y=mnist['data'],mnist['target']
X.shape

(70000, 784)

In [3]:
X = X/255

In [4]:
y = y.astype(np.uint8)
y = y[:,None] #keep y as a column vector

In [5]:
X_train,y_train = X[:60000],y[:60000]
X_test,y_test = X[60000:], y[60000:]

In [6]:
y_5_train = np.zeros((len(y_train),1))
y_5_train[y_train==0]=1

## normal equations

In [10]:
X_train.shape

(60000, 784)

In [11]:
X_train.max()

1.0

In [12]:
C = 1
M = X_train.T.dot(X_train) + C*np.identity(784)
b = X_train.T.dot(y_5_train)

In [13]:
theta = np.linalg.solve(M,b)

In [14]:
y_5_test = np.zeros((len(y_test),1))
y_5_test[y_test==0]=1

In [15]:
y_5_test_pred = X_test.dot(theta)

In [16]:
y_5_test_pred[y_5_test_pred>=0.5] = 1
y_5_test_pred[y_5_test_pred<0.5] = 0

In [17]:
'confusion matrix'
C = np.zeros((2,2))
for i in range(2):
    for j in range(2):
        C[i,j] = sum(y_5_test_pred[y_5_test==i]==j)
C.astype(int)

array([[8981,   39],
       [ 171,  809]])

In [18]:
y_test[3]

array([0], dtype=uint8)

In [20]:
X_new = X_test[3]

In [21]:
y_pred = X_new.dot(theta)

In [22]:
y_pred

array([0.84277884])

In [23]:
y_test[0]

array([7], dtype=uint8)

In [28]:
X_new = X_test[2]

In [29]:
y_pred = X_new.dot(theta)
y_pred

array([0.00215575])

## Gradient Descent